In [0]:
%pip install pymongo

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
%sql
select * from `e-commerce-project`.bronze.orders limit 1

order_id,customer_id,payment_method,order_status,event_time,shipping_city,shipping_pincode,ip_address,items,event_date
ORD004,C001,CARD,PLACED,2025-10-11T12:00:00Z,Pune,411001,203.0.113.11,"List(List(P003, 2, 1500.0, 3000.0), List(P009, 1, 3300.0, 3300.0))",2025-10-11


In [0]:
#Read Silver Orders
silver_orders = spark.table("`e-commerce-project`.silver.orders")

#Read Bronze Payments (raw attempts)
bronze_payments = spark.table("`e-commerce-project`.bronze.payments")

silver_payments = spark.table("`e-commerce-project`.silver.shipments").select("shipment_id","order_id","courier","dispatched_at","delivered_at","status","delivery_delay_hours","is_delivered","is_canceled")


#Detect Multiple Payment Attempts
from pyspark.sql import functions as F, Window

window = Window.partitionBy("order_id").orderBy("payment_timestamp")

multi_payments = bronze_payments \
    .withColumn("attempt_number", F.row_number().over(window)) \
    .filter(F.col("attempt_number") > 1) \
    .select("order_id", "customer_id", "method", "payment_timestamp")

# Join suspicious payments with Silver orders
enriched_fraud_df = silver_orders.join(
    multi_payments.select("order_id").distinct(),
    on="order_id",
    how="left"
    )\
    .join(
        silver_payments,
        on="order_id",
        how="left"
    ) \
    .withColumn(
    "has_multiple_payments", F.when(F.col("order_id").isin([x["order_id"] for x in multi_payments.collect()]), True).otherwise(False)
)

#Filter fraud candidates
fraud_df = enriched_fraud_df.filter(
    "fraud_score > 20 OR is_high_value = true OR has_multiple_payments = true"
)

from pyspark.sql import functions as F

# Define the item struct
fraud_orders_grouped = (
    fraud_df
    .withColumn(
        "item_struct",
        F.struct(
            F.col("product_id").alias("product_id"),
            F.col("quantity").alias("quantity"),
            F.col("price").alias("unit_price"),
            F.col("item_value_in_order").alias("amount")
        )
    )
    .groupBy(
        "order_id",
        "customer_id",
        "payment_method",
        "order_status",
        "event_time",
        "shipping_city",
        "shipping_pincode",
        "ip_address",
        "order_date",
        "order_total_value",
        "is_high_value",
        "fraud_score",
        "event_date",
        "shipment_id",
        "courier",
        "dispatched_at",
        "delivered_at",
        "status",
        "delivery_delay_hours",
        "is_delivered",
        "is_canceled",
        "has_multiple_payments"
    )
    .agg(
        F.collect_list("item_struct").alias("items")
    )
)
fraud_orders_grouped.drop(
    "product_id", "quantity", "price", "item_value_in_order"
)
fraud_orders_grouped.printSchema()
fraud_orders_grouped = fraud_orders_grouped.withColumn(
    "total_items", F.size("items")
)


# Upsert to MongoDB Atlas
from pymongo import MongoClient
mongo_uri = dbutils.secrets.get("mongodb", "atlas-uri")
client = MongoClient(mongo_uri)
db = client["ecommerce_project"]
collection = db["suspicious_orders"]

import datetime
import numpy as np

def convert_dates_and_arrays(obj):
    if isinstance(obj, dict):
        return {k: convert_dates_and_arrays(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_dates_and_arrays(i) for i in obj]
    elif isinstance(obj, np.ndarray):
        return [convert_dates_and_arrays(i) for i in obj.tolist()]
    elif isinstance(obj, (datetime.date, datetime.datetime)):
        return obj.isoformat()
    else:
        return obj

data = fraud_orders_grouped.toPandas().to_dict("records")
data = [convert_dates_and_arrays(record) for record in data]

for record in data:
    collection.update_one(
        {"order_id": record["order_id"]},
        {"$set": record},
        upsert=True
    )

# display(fraud_df.schema)

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- event_time: string (nullable = true)
 |-- shipping_city: string (nullable = true)
 |-- shipping_pincode: string (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- order_total_value: double (nullable = true)
 |-- is_high_value: boolean (nullable = true)
 |-- fraud_score: integer (nullable = true)
 |-- event_date: date (nullable = true)
 |-- shipment_id: string (nullable = true)
 |-- courier: string (nullable = true)
 |-- dispatched_at: timestamp (nullable = true)
 |-- delivered_at: timestamp (nullable = true)
 |-- status: string (nullable = true)
 |-- delivery_delay_hours: double (nullable = true)
 |-- is_delivered: boolean (nullable = true)
 |-- is_canceled: boolean (nullable = true)
 |-- has_multiple_payments: boolean (nullable = false)
 |-- items: arr

In [0]:
display(fraud_orders_grouped)

order_id,customer_id,payment_method,order_status,event_time,shipping_city,shipping_pincode,ip_address,order_date,order_total_value,is_high_value,fraud_score,event_date,shipment_id,courier,dispatched_at,delivered_at,status,delivery_delay_hours,is_delivered,is_canceled,has_multiple_payments,items,total_items
ORD008,C007,COD,PLACED,2025-10-15T16:00:00Z,Mumbai,400002,203.0.113.17,2025-10-15,4150.0,false,5,2025-10-15,SHIP_ORD008_3,EcomExpress,2025-10-16T07:30:00.000Z,2025-10-20T10:20:00.000Z,DELIVERED,98.83,true,false,true,"List(List(P014, 3, 900.0, 2700.0), List(P015, 1, 1450.0, 1450.0))",2
ORD015,C005,COD,PLACED,2025-10-30T17:45:00Z,Kolkata,700002,203.0.113.15,2025-10-30,25000.0,true,30,2025-10-30,SHIP_ORD015_3,DTDC,2025-10-31T09:00:00.000Z,null,CANCELED,null,false,true,false,"List(List(P001, 1, 25000.0, 25000.0))",1
ORD016,C006,COD,PLACED,2025-10-12T10:00:00Z,Delhi,110001,203.0.113.16,2025-10-12,15000.0,true,30,2025-10-12,SHIP_ORD016_1,BlueDart,2025-10-13T12:00:00.000Z,null,DISPATCHED,null,false,false,true,"List(List(P005, 1, 15000.0, 15000.0))",1
ORD003,C003,UPI,PLACED,2025-10-10T11:30:00Z,Bengaluru,560001,203.0.113.13,2025-10-10,55000.0,true,5,2025-10-10,SHIP_ORD003_3,DTDC,2025-10-11T07:30:00.000Z,2025-10-12T18:00:00.000Z,DELIVERED,34.5,true,false,false,"List(List(P002, 1, 55000.0, 55000.0))",1
ORD010,C009,CARD,PLACED,2025-10-17T10:30:00Z,Pune,411018,203.0.113.19,2025-10-17,57400.0,true,10,2025-10-17,SHIP_ORD010_3,Delhivery,2025-10-18T08:45:00.000Z,2025-10-20T18:10:00.000Z,DELIVERED,57.42,true,false,false,"List(List(P002, 1, 55000.0, 55000.0), List(P007, 3, 800.0, 2400.0))",2
ORD017,C007,CARD,PLACED,2025-10-12T10:05:00Z,Delhi,110001,203.0.113.16,2025-10-12,15000.0,true,10,2025-10-12,SHIP_ORD017_1,BlueDart,2025-10-12T10:10:00.000Z,2025-10-13T18:00:00.000Z,DELIVERED,31.83,true,false,true,"List(List(P005, 1, 15000.0, 15000.0))",1
ORD001,C001,COD,PLACED,2025-10-08T09:10:00Z,Mumbai,400001,203.0.113.11,2025-10-08,25950.0,true,30,2025-10-08,SHIP_ORD001_3,BlueDart,2025-10-09T08:00:00.000Z,2025-10-10T12:30:00.000Z,DELIVERED,28.5,true,false,true,"List(List(P001, 1, 25000.0, 25000.0), List(P008, 1, 950.0, 950.0))",2
ORD020,C009,UPI,PLACED,2025-10-13T09:00:00Z,Kolkata,700003,203.0.113.19,2025-10-13,20000.0,true,5,2025-10-13,SHIP_ORD020_1,EcomExpress,2025-10-14T09:00:00.000Z,2025-10-18T09:00:00.000Z,DELIVERED,96.0,true,false,false,"List(List(P010, 1, 20000.0, 20000.0))",1
ORD012,C002,COD,PLACED,2025-10-19T12:20:00Z,Chennai,600002,203.0.113.12,2025-10-19,3650.0,false,5,2025-10-19,SHIP_ORD012_3,BlueDart,2025-10-20T09:00:00.000Z,2025-10-21T10:30:00.000Z,DELIVERED,25.5,true,false,true,"List(List(P008, 2, 950.0, 1900.0), List(P011, 1, 1750.0, 1750.0))",2
ORD005,C004,COD,PLACED,2025-10-12T08:45:00Z,Chennai,600001,203.0.113.14,2025-10-12,5600.0,false,5,2025-10-12,SHIP_ORD005_3,BlueDart,2025-10-13T09:00:00.000Z,2025-10-14T17:45:00.000Z,DELIVERED,32.75,true,false,true,"List(List(P007, 2, 800.0, 1600.0), List(P010, 1, 4000.0, 4000.0))",2


----------------------------------------------------------------------

In [0]:
%sql
describe `e-commerce-project`.bronze.orders


col_name,data_type,comment
order_id,string,null
customer_id,string,null
payment_method,string,null
order_status,string,null
event_time,string,null
shipping_city,string,null
shipping_pincode,string,null
ip_address,string,null
items,array>,null
event_date,date,null
